In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
import pathlib
from tensorflow.keras.utils import image_dataset_from_directory
from collections import Counter


dataset_dir = '../../data/'

train_dir = dataset_dir + 'train/'
valid_dir = dataset_dir + 'valid/'
test_dir = dataset_dir + 'test/'
train_dataset = image_dataset_from_directory(
      train_dir,
      image_size=(180, 180),
      batch_size=32)


Found 84635 files belonging to 525 classes.


In [10]:
def get_top_tags():      
  train_labels = np.concatenate([y for _, y in train_dataset])
  top_tags = [tag for tag, _ in Counter(train_labels).most_common(3)]
  included_classes = [str(tag) for tag in top_tags]
  return included_classes

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from collections import Counter

# Create an ImageDataGenerator
image_data_generator = ImageDataGenerator(rescale=1.0/255.0)

# Create a list of included classes
included_classes = get_top_tags()
# Create a single train_generator for all top tags
train_generator = image_data_generator.flow_from_directory(
    train_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    classes=included_classes
)

# Create data generators for validation and test
valid_generator = image_data_generator.flow_from_directory(
    valid_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    classes=included_classes
)

test_generator = image_data_generator.flow_from_directory(
    test_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    classes=included_classes
)

# Print class names
print(train_generator.class_indices)


Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
{'434': 0, '291': 1, '374': 2}


In [9]:
i = 0
for item in filtered_dataset:
   print('yes')

In [25]:
for batch_images, batch_labels in train_dataset.take(5):
    for i in range(len(batch_images)):
        image = batch_images[i].numpy()  # Convert tensor to NumPy array
        label = batch_labels[i].numpy()   # Convert tensor to NumPy array

        # Display the image
        plt.figure()
        plt.imshow(image.astype("uint8"))
        plt.title(f"Class: {train_dataset.class_names[label]}")
        plt.axis("off")
        plt.show()

TypeError: '_BatchDataset' object is not subscriptable

In [2]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))

In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [5]:
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

1/1 [==============================] - 0s 130ms/step


In [6]:
train_features.shape

(84635, 5, 5, 512)

In [7]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 5, 5, 512)]       0         
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 256)               3277056   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 3277313 (12.50 MB)
Trainable params: 3277313 (12.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model using TensorFlow's Keras API
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

# Define callbacks using TensorFlow's Keras API
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
      filepath="./models/feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")
]

# Train the model using TensorFlow's Keras API
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

Epoch 1/20
2643/2645 [============================>.] - ETA: 0s - loss: -7109808128.0000 - accuracy: 0.0021

ValueError: The following argument(s) are not supported with the native Keras format: ['options']

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
test_model = keras.models.load_model(
    "./models/feature_extraction.keras")
test_loss, test_acc = test_model.evaluate(x=test_features, y=test_labels)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)

In [ ]:
conv_base.summary()

In [ ]:
import tensorflow as tf

In [ ]:
conv_base.trainable = False
conv_base.summary()

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()